In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
df = pd.read_csv("rebill_rate.csv")
df.tail()

,customer_account_id,own_started_at,order_id,amount_usd,gross_amount_usd,month,offer,currency,payment_type,payment_method,...,week_date,date,geo,paid_count,retry_count,month_started_at,utm_source,trial_period,period,next_billing_date
87072,221336.0,2024-02-20 06:00:34+00:00,c5bc700b-f9ec-44fe-abb5-754de1065cfe,973.0,9.73,2024-08-01 00:00:00+00:00,1Week,USD,recurring,card,...,2024-08-26,2024-08-30,T1,4,4,2024-02-01 00:00:00+00:00,facebook,7,28,2024-09-27
87073,96071.0,2023-12-29 18:16:18+00:00,8f045a1e-b635-4926-9ff5-b624bcb75b42,749.0,7.49,2024-09-01 00:00:00+00:00,Monthly,USD,recurring,card,...,2024-09-16,2024-09-20,WW,8,4,2023-12-01 00:00:00+00:00,google,84,28,2024-10-18
87074,91254.0,2023-12-28 07:36:21+00:00,ed6b4090-8362-4ae9-9fb6-fa48e16b8729,749.0,7.49,2024-07-01 00:00:00+00:00,Monthly,USD,recurring,card,...,2024-07-15,2024-07-19,T1,6,4,2023-12-01 00:00:00+00:00,facebook,84,28,2024-08-16
87075,102522.0,2023-12-31 18:35:57+00:00,9b2bc5be-d76a-4128-b77f-faf0cfa55bd1,749.0,7.49,2024-08-01 00:00:00+00:00,Monthly,USD,recurring,card,...,2024-08-12,2024-08-16,WW,6,4,2023-12-01 00:00:00+00:00,facebook,84,28,2024-09-13
87076,154936.0,2024-01-15 11:00:19+00:00,1ccc960d-bfd4-4481-90ef-3a902132cf73,749.0,7.49,2024-08-01 00:00:00+00:00,Monthly,USD,recurring,card,...,2024-08-26,2024-08-30,T1,7,4,2024-01-01 00:00:00+00:00,google,84,28,2024-09-27


In [32]:
date_features = ["own_started_at", "month", "created_at", "week_date", "date", "month_started_at", "next_billing_date"]
for feature in date_features:
    df[feature] = pd.to_datetime(df[feature], errors="coerce")

In [33]:
df.dtypes

customer_account_id                float64
own_started_at         datetime64[ns, UTC]
order_id                            object
amount_usd                         float64
gross_amount_usd                   float64
month                  datetime64[ns, UTC]
offer                               object
currency                            object
payment_type                        object
payment_method                      object
card_brand                          object
week_day                            object
bin                                float64
channel                             object
mid                                 object
created_at             datetime64[ns, UTC]
geo_country                         object
card_country                        object
week_date                   datetime64[ns]
date                        datetime64[ns]
geo                                 object
paid_count                           int64
retry_count                          int64
month_start

In [48]:
relevent_data = df[
    (df["own_started_at"] >= (df["own_started_at"].max() - pd.DateOffset(months=6))) & 
    (df["offer"].isin(["1Week", "4Week", "12Week"])) &
    (df["payment_method"].isin(["applepay", "card"]))
]

columns = ["offer", "payment_method", "geo"]

results = []

for segment in columns:
    segment_df = relevent_data.groupby(segment).agg(
        total_revenue=("gross_amount_usd", "sum"),
        total_clients=("customer_account_id", "nunique")
    )
    segment_df[f"LTV"] = np.round(segment_df["total_revenue"] / segment_df["total_clients"], 2)
    results.append(segment_df)

segment_results = {columns[i]: results[i] for i in range(len(columns))}
average_ltvs = {segment: df[f"LTV"].mean() for segment, df in segment_results.items()}

print("Averate LTV for each segment:")
for key, value in average_ltvs.items():
    print(f"{key}: {value} USD")

Averate LTV for each segment:
offer: 35.57 USD
payment_method: 35.44 USD
geo: 35.065 USD


In [49]:
segment_results["offer"]

,total_revenue,total_clients,LTV
offer,,,
12Week,1.598074e+05,3874,41.25
1Week,3.761341e+05,14234,26.43
4Week,1.039328e+06,26627,39.03


In [50]:
segment_results["geo"]


,total_revenue,total_clients,LTV
geo,,,
T1,1.183825e+06,33485,35.35
WW,3.914443e+05,11254,34.78


In [51]:
segment_results["payment_method"]

,total_revenue,total_clients,LTV
payment_method,,,
applepay,685674.140097,18627,36.81
card,889595.621630,26109,34.07
